# From version 6 I have now included the activation income in the model.

In [ ]:
import gurobipy as gp
import pandas as pd
from code_map import new_meters, new_markets, Inputs
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl
from itertools import combinations, product


In [ ]:
consumption_data =pd.read_csv('../master-data/customers-data/added_type_and_comp.csv')
timeframe = Inputs.one_day
freq_data = Inputs.get_frequency_data(timeframe, '../master-data/frequency_data/2023-06')
power_meter_dict = new_meters.create_meter_objects(consumption_data, timeframe)
afrr_activation_up = Inputs.get_afrr_activation_data(tf = timeframe, afrr_directory = '../master-data/aFRR_activation/', direction = "Up")
afrr_activation_down = Inputs.get_afrr_activation_data(tf = timeframe, afrr_directory = '../master-data/aFRR_activation/', direction = "Down")


In [ ]:
H = Inputs.get_timestamps(timeframe)

# Define the sets
L = list(power_meter_dict.values())  # List of PowerMeter objects
M = new_markets.all_market_list  # List of ReserveMarket objects

markets_dict = {market.name: market for market in M}

# make a list of only the meters that have direction up or both
L_u = [meter for meter in L if meter.direction != 'down']
L_d = [meter for meter in L if meter.direction != 'up']

In [ ]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

In [ ]:
Fu_h_l = np.array([[load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] if load.direction != "down" else 0 for load in L] for hour in H]) # set of flex volumes for meters, if load.direction != "down"
Fd_h_l = np.array([[load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] if load.direction != "up" else 0 for load in L] for hour in H]) # set of flex volumes for meters, if load.direction != "up"

R_h_l = np.array([[load.response_time for load in L]] * len(H)) # set of response times for meters

P_h_m = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of prices for markets
Vp_h_m = np.array([[market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of volumes for markets

Vm_m = [market.min_volume for market in M] # set of min values for markets
R_m = [market.response_time for market in M] # set of response times for markets


In [ ]:
np.sum(Fu_h_l)

In [ ]:
total_up_flex = np.sum(Fu_h_l) # total available flex volume up
total_down_flex = np.sum(Fd_h_l) # total available flex volume down
#total_flex = total_up_flex + total_down_flex
hourly_flex_up = total_up_flex/len(H)
hourly_flex_down = total_down_flex/len(H)

print(f"Total up flex volume: {total_up_flex}")
print(f"Total down flex volume: {total_down_flex}")
print(f"Average flex volume pr hour up: {hourly_flex_up}")
print(f"Average flex volume pr hour down: {hourly_flex_down}")

In [ ]:
def get_dominant_direction(freq_df : pd.DataFrame, hour : pd.Timestamp):
    """will find out which direction is dominant within an hour

    Args:
        freq_data (pd.DataFrame): dataframe of the frequency data
        hour (pd.Timestamp): the wanted hour
    """
    start_datetime = hour 
    end_datetime = hour + pd.Timedelta(hours=1)
        
    filtered_df = freq_df[(freq_df["Time"] >= start_datetime) & (freq_df["Time"] <= end_datetime)]
    #print(filtered_df)
    avg_freq = filtered_df["Value"].mean()
    #print(avg_freq)
    if avg_freq > 50.0:
        return "up"
    else:
        return "down"

In [ ]:
[m.name for m in M]

In [ ]:
 """elif "FCR_D" in market:
                FCR_D_up_activation =filtered_df.loc[(filtered_df["Value"] <= 49.9) & (filtered_df["Value"] >= 49.5)]
                return len(FCR_D_up_activation)/len(filtered_df), 0
            
            elif "FFR" in market:
                FFR_activation = filtered_df.loc[(filtered_df["Value"] < 49.7)]
                return len(FFR_activation)/len(filtered_df), 0
            else:
                return 0,0"""

In [ ]:
def get_activation_percentages(freq_df : pd.DataFrame, timeframe, markets):
    """ Get a dictionary of the activation percentages for each market and hour

    Args:
        freq_df (pd.DataFrame): dataframe of the frequency data
        timeframe (list): list of the wanted hours
        markets (list): list of the markets

    Returns:
        dict: a dictionary of the activation percentages for each market and hour
    """
    
    freq_dict = {}
    for h, hour in enumerate(timeframe):
        start_datetime = hour 
        end_datetime = hour + pd.Timedelta(hours=1)
        for m, market in enumerate(markets):
            filtered_df = freq_df[(freq_df["Time"] >= start_datetime) & (freq_df["Time"] <= end_datetime)]
            if "FCR_N" in market.name:
                FCR_N_up_activation = filtered_df.loc[(filtered_df["Value"] > 49.9) & (filtered_df["Value"] < 50.0)]
                FCR_N_down_activation = filtered_df.loc[(filtered_df["Value"] < 50.1) & (filtered_df["Value"] > 50.0)]
                freq_dict[h,m] = (len(FCR_N_up_activation)/len(filtered_df), len(FCR_N_down_activation)/len(filtered_df))
            else:
                freq_dict[h,m] = (0,0)
    return freq_dict

In [ ]:
F = get_activation_percentages(freq_data, H, M)

In [ ]:
dominant_directions = [get_dominant_direction(freq_data, hour) for hour in H]

In [ ]:
dominant_directions

In [ ]:
Ir = {}
Ia = {}

for h, hour in enumerate(H):
    for l, load in enumerate(L): 
        for m, market in enumerate(M):
            if market.direction == "both":
                if load.direction == "both":
                    if dominant_directions[h] == "up":
                        Ir[h,l,m] = Fu_h_l[h,l] * P_h_m[h,m]
                    else:
                        Ir[h,l ,m] = Fd_h_l[h,l] * P_h_m[h,m]
                    #I[h,l,m] =(Fu_h_l[h,l]+ Fd_h_l[h,l])/2 * P_h_m[h,m]
                else:
                    Ir[h,l,m] = 0
            elif market.direction == "up":
                if load.direction != "down":
                    Ir[h,l,m] = Fu_h_l[h,l] * P_h_m[h,m]
                else:
                    Ir[h,l,m] = 0
            else: # market.direction == "down"
                if load.direction != "up":
                    Ir[h,l,m] = Fd_h_l[h,l] * P_h_m[h,m]
                else:
                    Ir[h,l,m] = 0
            if market.capacity_market: 
                if "FCR_N" in market.name:
                    up_val, down_val = F[h,m]
                    activation_income = (up_val * markets_dict["RK_up_" + market.area].price_data.loc[markets_dict["RK_up_" + market.area].price_data["Time(Local)"] == hour].values[0][1] + 
                                         down_val * markets_dict["RK_down_" + market.area].price_data.loc[markets_dict["RK_down_" + market.area].price_data["Time(Local)"] == hour].values[0][1])
                    # Add to the objective expression
                    Ia[h,l,m] = activation_income
                else: # will have to add the other markets later - especially aFRR and RKOM
                    # No activation income, just regular income
                    Ia[h,l,m] = 0
            else:
                # No capacity market, just regular income
                Ia[h,l,m] = 0
        

In [ ]:
"""else:
        continue"""

compatible_list = []
for h, hour in enumerate(H):
    hour_list = []
    for l, asset in enumerate(L):
        asset_list = []
        for m, market in enumerate(M):
            if asset.direction == "up":
                if market.direction == "up":
                    if market.area == asset.area or market.area == "all":
                        asset_list.append(m)
            elif asset.direction == "down":
                if market.area == asset.area or market.area == "all":
                    if market.direction == "down":
                        asset_list.append(m)
                
            elif asset.direction == "both":
                if market.area == asset.area  or market.area == "all":
                    asset_list.append(m)
        hour_list.append(asset_list)
    compatible_list.append(hour_list)

                
    
    

In [ ]:
markets_dict["RK_up_" + M[4].area].price_data.loc[markets_dict["RK_up_" + M[4].area].price_data["Time(Local)"] == H[3]].values[0][1]

In [ ]:
# Create a new model
test_model = gp.Model("AssetToMarket")

# Create decision variables
x = {}
d = {}
y = {}
w = {}
for h in range(len(H)):
    for l in range(len(L)):
        for m in range(len(M)):
            # asset i is connected to market j at hour h
            x[h, l, m] = test_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY, name=f"x_{h}_{l}_{m}")

            d[h,l,m] = 1 if m in compatible_list[h][l] else 0 # compatible_list takes care of both the area constraint and the direction constraint
            
            # adding the constraint
            test_model.addConstr(x[h,l,m] <= d[h,l,m])
    for m in range(len(M)):
        # market j is active at hour h
        y[h, m] = test_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY, name=f"y_{h}_{m}")
        w[h, m] = test_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY , name=f"w_{h}_{m}")
        

# Set objective            

# Set the objective to maximize the total income expression
test_model.setObjective(sum(Ir[h,l,m] * x[h,l,m] + Ia[h,l,m] * w[h,m] for h in range(len(H)) for l in range(len(L)) for m in range(len(M)))
                        , gp.GRB.MAXIMIZE)

# Add constraints
for h in range(len(H)):
    for l in range(len(L)):
        # Each asset can only be connected to one market at a time
        test_model.addConstr(sum(x[h, l, m] for m in range(len(M))) <= 1, f"single_market_for_asset_at_hour_{h}_nr.{l}")

    for m in range(len(M)):
        # Connect the binary variables by using big M
        test_model.addConstr((sum(x[h, l, m] for l in range(len(L)))) <= len(L) * y[h, m], f"asset_connection_for_hour_{h}_market_{m}")
        
        # Min volume constraint
        if market.direction == "up":
            test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes
        elif market.direction == "down":
            test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes
        else:
            if dominant_directions[h] == "up":
                test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes
            else:
                test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes

     
        # Max volume constraint
        
        if market.direction == "up":
            test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
        elif market.direction == "down":
            test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
        else:
            if dominant_directions[h] == "up":
                test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
            else:
                test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
        
        up_val, down_val = F[h,m]
        if up_val + down_val > 0:
            test_model.addConstr(w[h,m] <= y[h,m], f"market_{m}_can_not_be_activated_at_hour_{h}_if_it_is_not_active")
        else:
            test_model.addConstr(w[h,m] == 0, f"market_{m}_can_not_be_activated_at_hour_{h}_if_it_is_not_active")
        
        
        
        
        for l in range(len(L)):
            test_model.addConstr(x[h,l,m] * R_h_l[h,l] <= R_m[m] * y[h,m], f"response_time_for_hour_{h}_market_{m}")

# Solve the model
test_model.optimize()
    
if test_model.status == gp.GRB.Status.INFEASIBLE:
    test_model.computeIIS()


In [ ]:
def get_market_count_dict(x):
    data = []

    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            for m, market in enumerate(M):
                if x[h, l, m].X > 0.5:
                    data.append([hour, load.meter_id, market.name])

    df = pd.DataFrame(data, columns=["Hour", "Asset Meter ID", "Market"])
    #print(df)


    market_count_dict = {}
    for hour in H:
        hour_df = df.loc[(df["Hour"] == hour)]
        # get the number of assets in each market
        market_count = hour_df.groupby(["Market", "Hour"]).agg("count").reset_index().rename(columns={"Asset Meter ID": "Asset Count"})
        market_count_dict[hour] = market_count
    return market_count_dict

In [ ]:
market_count_dict = get_market_count_dict(x)


In [ ]:
market_count_dict

In [ ]:
def test_solution_validity(x, y, L, M, H):
    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            # Each asset can only be connected to one market at a time
            assert sum(x[h, l, m].X for m in range(len(M))) <= 1, f"Asset {l} connected to multiple markets at hour {h}"
            for m, market in enumerate(M):
                # Directionality constraints
                if load.direction == "up" and market.direction == "down":
                    assert x[h, l, m].X == 0, f"Up-direction asset {l} connected to down-direction market {m} at hour {h}"
                elif load.direction == "down" and market.direction == "up":
                    assert x[h, l, m].X == 0, f"Down-direction asset {l} connected to up-direction market {m} at hour {h}"
                elif market.direction == "both" and load.direction != "both":
                    assert x[h, l, m].X == 0, f"Asset {l} with specific direction connected to both-direction market {m} at hour {h}"
                elif market.area != load.area:
                    assert x[h, l, m].X == 0, f"Asset {l} in area {load.area} connected to market {m} in area {market.area} at hour {h}"
                
                # Response time constraints
                assert x[h, l, m].X * load.response_time <= market.response_time * y[h, m].X, f"Asset {l} connected to market {m} at hour {h} violates response time constraint"
                
        for m, market in enumerate(M):
            # Connect the binary variables by using big M
            assert sum(x[h, l, m].X for l in range(len(L))) <= len(L) * y[h, m].X, f"More than allowed assets connected at hour {h} to market {m}"

            #total_flex_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))

            # Min volume constraint
            if market.direction == "up":
                total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
            elif market.direction == "down":
                total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            else:
                if dominant_directions[h] == "up":
                    total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
                    
            assert total_flex_volume >= market.min_volume * y[h, m].X, f"Minimum volume constraint violated at hour {h} for market {m}"

            # test_model.addConstr(sum(x[h, l, m] * F_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes
            # F_h_l = np.array([[meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0] for meter in L] for hour in H]) # set of flex volumes for meters


            # Weighted average constraint for response time
            """total_response_time = sum(x[h, l, m].X * load.response_time for l, load in enumerate(L))
            total_assigned_assets = sum(x[h, l, m].X for l in range(len(L)))
            assert total_response_time <= market.response_time * total_assigned_assets, f"Average response time constraint violated at hour {h} for market {m}"""
            
            # Max volume constraint
            if market.direction == "up":
                total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
            elif market.direction == "down":
                total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            else:
                if dominant_directions[h] == "up":
                    total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
                
            market_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
            assert total_max_volume <= market_max_volume * y[h,m].X, f"Maximum volume constraint violated at hour {h} for market {m}"





In [ ]:
test_solution_validity(x, y, L, M, H)


In [ ]:
for h, hour in enumerate(H):
    for l, load in enumerate(L):
        if load.direction != "down":
            if load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] != Fu_h_l[h,l]:
                print("ERROR")
                print(load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0], Fu_h_l[h,l])
        elif load.direction != "up":
            if load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] != Fd_h_l[h,l]:
                print("ERROR")
                print(load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0], Fd_h_l[h,l])
                
for m, market in enumerate(M):
    if Vm_m[m] != market.min_volume:
        print("ERROR")
        print(Vm_m[m], market.min_volume)

In [ ]:
#h = 0
m = 26
for hour in range(len(H)):
    total_up_flex_volume_test = [x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == H[h]].values[0] for l, load in enumerate(L) if load.direction != "down"]
    total_up_flex_volume_mod = [x[h, l, m].X * Fu_h_l[h,l] for l, load in enumerate(L)]
    test_sum_up = sum(total_up_flex_volume_test)
    mod_sum_up = sum(total_up_flex_volume_mod)
    print(f" the up sums are eaqual : {test_sum_up == mod_sum_up}")
    #print(f"all the values are eaqual : {all(total_flex_volume_test == total_flex_volume_mod)}")
    
    total_down_flex_volume_test = [x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == H[h]].values[0] for l, load in enumerate(L) if load.direction != "up"]
    total_down_flex_volume_mod = [x[h, l, m].X * Fd_h_l[h,l] for l, load in enumerate(L)]
    test_sum_down = sum(total_down_flex_volume_test)
    mod_sum_down = sum(total_down_flex_volume_mod)
    print(f" the down sums are eaqual : {test_sum_down == mod_sum_down}")


"""print(f"The flex volume list for the test for market {m} at hour {h} is {total_flex_volume_test}")
print(f"The flex volume list for the model for market {m} at hour {h} is {total_flex_volume_mod}")
print(f"The total flex volume in the test for market {m} at hour {h} is {test_sum}")
print(f"The total flex volume in the model for market {m} at hour {h} is {mod_sum}")"""
    